In [57]:
# Carga de librerías y lectura del archivo que contiene los datos

import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline

from google.colab import files

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
from joblib import dump, load
# Cargar el modelo desde el archivo
loaded_svm_model = load('/content/drive/MyDrive/proyecto/anova_svm_model.joblib')


In [60]:
pip install --upgrade scikit-learn

#Carga de modelos supervisados

In [61]:
loaded_knn_model = load('/content/drive/MyDrive/proyecto/anova_knn_model.joblib')

In [62]:

loaded_rf_model = load('/content/drive/MyDrive/proyecto/anova_rf_model.joblib')

#Carga de modelos no supervisados

In [99]:
loaded_red_base = load('/content/drive/MyDrive/proyecto/CNN_base_model.joblib')

In [98]:
loaded_red_vgg16 = load('/content/drive/MyDrive/proyecto/VGG16_base_model.joblib')

#Carga de data - transcriptomas

In [64]:
integracion_Data = pd.read_excel('/content/drive/MyDrive/proyecto/transciptomas_integracion.xlsx',header = 0, na_values= None)
integracion_Data.shape

(2, 14193)

In [65]:
integracion_Data

,Outcome,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000286185,ENSG00000286231,ENSG00000286235,ENSG00000286239,ENSG00000286261,ENSG00000286905,ENSG00000287725,ENSG00000287856,ENSG00000288436,ENSG00000288547
0,1,7.115740,0.00000,33.792677,27.640822,6.236903,176.504366,406.277557,73.170217,20.723529,...,3.118452,0.737089,2.296314,5.386417,12.587205,3.203500,0.708739,8.419820,0.793788,0.311845
1,0,9.493824,0.07978,36.299914,23.375549,6.382402,50.181639,124.696188,50.101859,53.053720,...,0.957360,1.515821,0.877580,1.755161,5.504822,2.712521,0.877580,23.375549,1.037140,0.398900


In [66]:
Datadeseq = pd.read_excel('/content/drive/MyDrive/deseq_cur_fall_v1_v2.xlsx', na_values= None)
Datadeseq

,row.names
0,ENSG00000156282
1,ENSG00000171864
2,ENSG00000196344
3,ENSG00000171403
4,ENSG00000197915
...,...
166,ENSG00000205846
167,ENSG00000131203
168,ENSG00000125144
169,ENSG00000205358


In [67]:
integracion_Data = integracion_Data.apply(pd.to_numeric, errors='coerce')

In [68]:
col_select = Datadeseq['row.names'].values
int_DataNew = integracion_Data[col_select]
int_DataNew

,ENSG00000156282,ENSG00000171864,ENSG00000196344,ENSG00000171403,ENSG00000197915,ENSG00000181092,ENSG00000171195,ENSG00000096006,ENSG00000155897,ENSG00000161798,...,ENSG00000162892,ENSG00000137757,ENSG00000138755,ENSG00000108700,ENSG00000205364,ENSG00000205846,ENSG00000131203,ENSG00000125144,ENSG00000205358,Outcome
0,0.00000,0.00000,0.028350,0.198447,0.198447,0.00000,0,0.00000,0.00000,0.000000,...,36.627633,30.390729,7249.719800,515.933658,19.532847,9.723899,1450.250135,112.009115,71.497592,1
1,0.07978,0.95736,8.057783,7.738663,395.389831,0.07978,0,0.63824,0.31912,10.850084,...,1.834941,5.504822,2429.541269,52.176140,1.834941,2.473181,168.654984,4.387902,1.675381,0


In [69]:
X = int_DataNew.drop(['Outcome'],axis = 1)
y = int_DataNew["Outcome"]

In [70]:
X_test = X.iloc[:, [22,24,41]]

In [71]:
X_test

,ENSG00000166211,ENSG00000198542,ENSG00000177098
0,0.02835,0.028350,0.850487
1,0.07978,3.829441,14.679526


#carga de imagenes

In [72]:
import os

import numpy as np
from PIL import Image
def load_images_from_dir(directory):
    # Initialize empty lists for images and labels
    images = []
    labels = []

    # Leemos las imagenes en el directorio
    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # Load image and convert to rgb
            img = Image.open(os.path.join(directory, filename)).convert('RGB')
            if img.size != (224, 224):
                img = img.resize((224, 224))
            img_array = np.array(img)

            # Add image and label to lists
            images.append(img_array)
            labels.append(filename.split('-')[0])

    # Convert lists to numpy arrays
    images_array = np.array(images, dtype=np.uint8)
    labels_array = np.array(labels)

    return images_array, labels_array

In [73]:
integracion_image = '/content/drive/MyDrive/proyecto/Fotosintegracion'
int_X, int_Y = load_images_from_dir(integracion_image)

In [74]:
X_test_i = int_X.astype('float32')

In [75]:
X_test_i = int_X / 255.

#Integración con votación

In [109]:
import numpy as np

umbral = 0.5
# Supongamos que tienes predicciones de tus modelos para un conjunto de datos de prueba X_test
preds_model_1 = loaded_svm_model.predict(X_test)
preds_model_2 = loaded_knn_model.predict(X_test)
preds_model_3 = loaded_rf_model.predict(X_test)
preds_red_1 = loaded_red_base.predict(X_test_i)
preds_red_1 = (preds_red_1 > umbral).astype(int)
preds_red_1 = [item for sublist in preds_red_1 for item in sublist]
preds_red_2 = loaded_red_vgg16.predict(X_test_i)
preds_red_2 = (preds_red_2 > umbral).astype(int)
preds_red_2 = [item for sublist in preds_red_2 for item in sublist]



# Realizar una votación por mayoría (cuenta de votos)
voting_result = np.sum([preds_model_1, preds_model_2, preds_model_3, preds_red_1, preds_red_2], axis=0)  # Suma las predicciones

#Obtener la predicción final basada en la votación

final_prediction = np.round(voting_result / 5)  # Promedio de las predicciones
final_prediction


1/1 [==============================] - 1s 1s/step


array([1., 0.])

In [110]:
voting_result

array([5, 2])

In [112]:
accuracy = accuracy_score(y, final_prediction)
accuracy

1.0